In [1]:
%pip install deep_translator geopandas

  Obtaining dependency information for deep_translator from https://files.pythonhosted.org/packages/38/3f/61a8ef73236dbea83a1a063a8af2f8e1e41a0df64f122233938391d0f175/deep_translator-1.11.4-py3-none-any.whl.metadata
  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/c4/64/7d344cfcef5efddf9cf32f59af7f855828e9d74b5f862eddf5bfd9f25323/geopandas-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyogrio>=0.7.2 from https://files.pythonhosted.org/packages/a0/f6/564e90bc8a07cb4895e15e396bc10aeb214a369cd9594f67672c93e1c111/pyogrio-0.9.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pyproj>=3.3.0 from https://files.pythonhosted.org/packages/97/0a/b1525be9680369cc06dd288e12c59d24d5798b4afcdcf1b0915836e1caa6/pyproj-3.6.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for shapely>=2.0.0 from https://files.pythonhosted.org/p

In [2]:
import mlrun
import os
import pandas as pd

In [3]:
project = mlrun.get_or_create_project("datiprotezione", context="./")

> 2024-07-29 09:23:06,773 [info] Project loaded successfully: {'project_name': 'datiprotezione'}


In [4]:
new_folder ='src'
if not os.path.exists(new_folder):
    os.makedirs(new_folder)


In [5]:
%%writefile "src/create-list.py"

import requests
import pandas as pd
from deep_translator import GoogleTranslator
import json
import mlrun

# translation
translator = GoogleTranslator(source='it', target='en')
def translate_text(text):
    if text is not None:
        text = str(text)
        text = translator.translate(str(text))
    return text

def givemelink(x):
    datakeywords = ['siat','urbanistica-dati','catasto']
    urlitems = []
    rd = x
    if isinstance(x, list):
        for urlist in x:
            for datakey in datakeywords:
                if datakey.lower() in urlist.lower():
                    urlitems = urlist.split("||")
                    break
    else:
        urlitems = x.split("||")
    
    for urlitem in urlitems:
        for datakey in datakeywords:
            if datakey.lower() in urlitem.lower():
                rd = urlitem
                break

    return(rd.replace("||",""))

@mlrun.handler()
def create_list(context):
    start = 1
    to = 20
    maxPageSize = 100
    url = "https://siat.provincia.tn.it/geonetwork/srv/ita/q?_content_type=json&any=&bucket=s101&facet.q=&fast=index&from=START&resultType=details&sortBy=relevance&sortOrder=&to=TO" 
    url = url.replace("START",str(start)).replace("TO",str(to))
    response = requests.get(url)
    data = response.json()
    start = int(data['@from'])
    to = int(data['@to'])
    total = int(data['summary']['@count'])
    rest = total % maxPageSize
    steps = (total - rest) / 100    
    
    dfs = []
    for start in range(int(steps)):
        start = 1 + start * 100
        to = start + 100 -1
        url = "https://siat.provincia.tn.it/geonetwork/srv/ita/q?_content_type=json&any=&bucket=s101&facet.q=&fast=index&from=START&resultType=details&sortBy=relevance&sortOrder=&to=TO" 
        url = url.replace("START",str(start)).replace("TO",str(to))
        response = requests.get(url)
        data = response.json()
        records = data.get('metadata', [])
        df = pd.DataFrame(records)
        dfs.append(df)
    
    start = to +1
    to = start + rest -1
    url = "https://siat.provincia.tn.it/geonetwork/srv/ita/q?_content_type=json&any=&bucket=s101&facet.q=&fast=index&from=START&resultType=details&sortBy=relevance&sortOrder=&to=TO" 
    url = url.replace("START",str(start)).replace("TO",str(to))
    response = requests.get(url)
    data = response.json()
    records = data.get('metadata', [])
    df = pd.DataFrame(records)
    dfs.append(df)
    
    # create a dataset with all the entries
    siatdata = pd.concat(dfs, ignore_index=True)
    # because these kind of attributes contain in some cases list I extract only the first value
    # (change it if you want have more information)
    siatdata['type'] = siatdata['type'].apply(lambda x: x[0] if isinstance(x, list) else x) 
    siatdata['identifier'] = siatdata['identifier'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    siatdata['format'] = siatdata['format'].apply(lambda x: x[0] if isinstance(x, list) else x)    
    
    siatdata['link'] = siatdata['link'].apply(givemelink)
    
    data = siatdata[['title','abstract','lineage','resourceConstraints','type',
          'legalConstraints','identifier',"crsDetails","maintenanceAndUpdateFrequency_text",
          'spatialRepresentationType_text','denominator',
          'tempExtentBegin','tempExtentEnd','serviceType',
          'updateFrequency','revisionDate','classification_text','defaultTitle',
          'publicationDate','creationDate','crs',"parentId",'link']]
    
    data['title_en'] = data['defaultTitle'].apply(translate_text)
    
    context.log_dataset("siat_trentino", df=data, index=False)

Overwriting src/create-list.py


In [6]:
project.set_function("src/create-list.py", "create-list", kind="job", image="mlrun/mlrun", handler="create_list", requirements=["deep_translator", "geopandas"])
project.save()

In [ ]:
project.run_function("create-list", local=True)

In [7]:
di = mlrun.get_dataitem("s3://datalake/projects/datiprotezione/artifacts/create-list-create-list/0/siat_trentino.parquet")
df = di.as_df()

In [8]:
df

,title,abstract,lineage,resourceConstraints,type,legalConstraints,identifier,crsDetails,maintenanceAndUpdateFrequency_text,spatialRepresentationType_text,...,updateFrequency,revisionDate,classification_text,defaultTitle,publicationDate,creationDate,crs,parentId,link,title_en
0,Distribuzione potenziale della pernice bianca ...,Distribuzione potenziale della pernice bianca ...,Esprime la vocazionalità del territorio ad osp...,Creative Commons Attribuzione 4.0 Internaziona...,dataset,nessuna limitazione,p_TN:6847fc74-160d-40f6-8ae3-3871bd345043_reso...,"{'code': ':ETRS89-GRS80', 'name': '', 'url': '...",Non pianificato,Dati vettoriali,...,notPlanned,2010-01-01,Non riservato,Distribuzione potenziale della pernice bianca ...,2020-02-10,2010-12-30,ETRS89-GRS80,None,https://siatservices.provincia.tn.it/idt/vecto...,Potential distribution of the ptarmigan - Prov...
1,Distribuzione reale invernale del cervo - Pian...,Distribuzione reale invernale del cervo - Pian...,Individua l'areale di presenza della specie ri...,Creative Commons Attribuzione 4.0 Internaziona...,dataset,nessuna limitazione,p_TN:8149a088-494a-4f6b-8dd0-4ee7dbcdc599_reso...,"{'code': ':ETRS89-GRS80', 'name': '', 'url': '...",Non pianificato,Dati vettoriali,...,notPlanned,2010-01-01,Non riservato,Distribuzione reale invernale del cervo - Pian...,2020-02-10,2010-12-30,ETRS89-GRS80,None,https://siatservices.provincia.tn.it/idt/vecto...,Real winter distribution of the deer - Provinc...
2,Distribuzione potenziale invernale del capriol...,Distribuzione potenziale invernale del capriol...,Esprime la vocazionalità del territorio ad osp...,Creative Commons Attribuzione 4.0 Internaziona...,dataset,nessuna limitazione,p_TN:65fd7b3f-970d-4c11-9691-8963331ed264_reso...,"{'code': ':ETRS89-GRS80', 'name': '', 'url': '...",Non pianificato,Dati vettoriali,...,notPlanned,2010-01-01,Non riservato,Distribuzione potenziale invernale del capriol...,2020-02-10,2010-12-30,ETRS89-GRS80,None,https://siatservices.provincia.tn.it/idt/vecto...,Potential winter distribution of roe deer - Pr...
3,Distribuzione reale della pernice bianca- Pian...,Distribuzione reale della pernice bianca- Pian...,Individua l'areale di presenza della specie ri...,Creative Commons Attribuzione 4.0 Internaziona...,dataset,nessuna limitazione,p_TN:4b24a620-a57c-46eb-8a38-b0b2762497a4_reso...,"{'code': ':ETRS89-GRS80', 'name': '', 'url': '...",Non pianificato,Dati vettoriali,...,notPlanned,2010-01-01,Non riservato,Distribuzione reale della pernice bianca- Pian...,2020-02-10,2010-12-30,ETRS89-GRS80,None,https://siatservices.provincia.tn.it/idt/vecto...,Real distribution of the white ptarmigan - Pro...
4,Distribuzione reale del gallo cedrone - Piano ...,Distribuzione reale del gallo cedrone - Piano ...,Individua l'areale di presenza della specie ri...,Creative Commons Attribuzione 4.0 Internaziona...,dataset,nessuna limitazione,p_TN:422f917d-49bc-4609-8a15-8cdfef474fd5_reso...,"{'code': ':ETRS89-GRS80', 'name': '', 'url': '...",Non pianificato,Dati vettoriali,...,notPlanned,2010-12-30,Non riservato,Distribuzione reale del gallo cedrone - Piano ...,2020-02-10,2010-12-30,ETRS89-GRS80,None,https://siatservices.provincia.tn.it/idt/vecto...,Real distribution of the capercaillie - Provin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,Quadro d'unione LIDAR Campolongo 2011,Quadro d'unione delle tavole del rilievo LiDAR...,Il rilievo è stato effettuato dalla Ditta Heli...,Creative Commons Attribuzione 4.0 Internaziona...,dataset,nessuna limitazione,p_TN:bb58959b-aeb8-4eda-a039-59c73f8833f4,"{'code': ':ETRS89-GRS80', 'name': '', 'url': '...",Non pianificato,Dati vettoriali,...,notPlanned,2020-10-27,Non riservato,Quadro d'unione LIDAR Campolongo 2011,2020-10-27,2011-07-22,ETRS89-GRS80,None,https://siatservices.provincia.tn.it/idt/vecto...,LIDAR Union Framework Campolongo 2011
425,ambiti territoriali omogenei del camoscio,Rappresentano gli ambiti territoriali omogenei...,acque correnti - C

In [21]:
%%writefile "src/convert-list.py"

import geopandas
import mlrun
import os
import urllib.request
import pandas as pd

def read_geopandas(name, url, context):
    gdf = geopandas.read_file(url)
    gdf.to_parquet('./'+name+'.parquet')
    with open('./'+name+'.parquet', 'rb') as in_file:
        content = in_file.read()
        context.log_artifact(name, body=content, format="parquet", db_key=name)
        os.remove('./'+name+'.parquet')

def read_file(name, url, context):
    # get extension from url
    extension = url.split(".")[-1]
    urllib.request.urlretrieve(url, f"./{name}.{extension}")
    with open(f"./{name}.{extension}", "rb") as in_file:
        content = in_file.read()
        context.log_artifact(name, body=content, format=extension, db_key=name)
        os.remove(f"./{name}.{extension}")
        
@mlrun.handler()
def convert_list(context, list: mlrun.DataItem):
    data = list.as_df()
    data = data.to_dict(orient='records')
    error_data = []
    for index, item in enumerate(data):
        name = item['title_en']
        link = item['link']
        context.logger.info(f"Converting {name} to parquet (link {link})")
        # convert name to lower case and replace spaces with underscores
        name = name.lower().replace(" ", "_").replace("(", "").replace(")", "").replace("/", "_")
        if link.endswith(".zip"):
            try:
                read_geopandas(name, link, context)
            except:
                try:
                    read_file(name, link, context)
                except:
                    context.logger.error(f"Error reading file: {item['link']}")
                    continue
        elif link.endswith(".png"):
            try:
                read_file(name, link, context)
            except:
                context.logger.error(f"Error reading file: {item['link']}")
                continue
        else:    
            context.logger.error(f"Error reading file: {item['link']}")
            error_data.append(item)
            continue

    context.log_dataset("error_data", df=pd.DataFrame(error_data), index=False)

Overwriting src/convert-list.py


In [22]:
project.set_function("src/convert-list.py", "convert-list", kind="job", image="mlrun/mlrun", handler="convert_list", requirements=["geopandas"])
project.save()

In [23]:
project.build_function("convert-list", base_image="mlrun/mlrun")

> 2024-07-29 11:06:42,625 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2024-07-29 11:06:42,716 [info] Started building image: .digitalhub-test/func-datiprotezione-convert-list:latest


The `overwrite_build_params` parameter default will change from 'False' to 'True' in 1.8.0.
The clone_target_dir attribute is deprecated in 1.6.2 and will be removed in 1.8.0. Use spec.build.source_code_target_dir instead.


INFO[0000] Retrieving image manifest mlrun/mlrun:1.6.2-rc1 
INFO[0000] Retrieving image mlrun/mlrun:1.6.2-rc1 from registry index.docker.io 
INFO[0001] Built cross stage deps: map[]                
INFO[0001] Retrieving image manifest mlrun/mlrun:1.6.2-rc1 
INFO[0001] Returning cached image manifest              
INFO[0001] Executing 0 build triggers                   
INFO[0001] Unpacking rootfs as cmd RUN echo 'Installing /empty/requirements.txt...'; cat /empty/requirements.txt requires it. 
INFO[0036] RUN echo 'Installing /empty/requirements.txt...'; cat /empty/requirements.txt 
INFO[0036] Taking snapshot of full filesystem...        
INFO[0057] cmd: /bin/sh                                 
INFO[0057] args: [-c echo 'Installing /empty/requirements.txt...'; cat /empty/requirements.txt] 
INFO[0057] Running: [/bin/sh -c echo 'Installing /empty/requirements.txt...'; cat /empty/requirements.txt] 
Installing /empty/requirements.txt...
mlrun[complete]==1.6.2-rc1
geopandas
INFO[0057] Taking

BuildStatus(ready=True, outputs={'image': '.digitalhub-test/func-datiprotezione-convert-list:latest'})

In [24]:
project.run_function("convert-list", inputs={"list": "s3://datalake/projects/datiprotezione/artifacts/create-list-create-list/0/siat_trentino.parquet"}, local=False)

> 2024-07-29 11:08:02,183 [info] Storing function: {'name': 'convert-list-convert-list', 'uid': '5a838b7dcdc7420c9d6f41e199bbcc7d', 'db': 'http://mlrun-api:8080'}
> 2024-07-29 11:08:02,291 [info] Job is running in the background, pod: convert-list-convert-list-2ck28
> 2024-07-29 11:08:08,201 [info] Converting Potential distribution of the ptarmigan - Provincial Wildlife Plan 2010 to parquet (link https://siatservices.provincia.tn.it/idt/vector/p_TN_6847fc74-160d-40f6-8ae3-3871bd345043.zip)
> 2024-07-29 11:08:10,863 [info] Converting Real winter distribution of the deer - Provincial Wildlife Plan 2010 to parquet (link https://siatservices.provincia.tn.it/idt/vector/p_TN_8149a088-494a-4f6b-8dd0-4ee7dbcdc599.zip)
> 2024-07-29 11:08:11,245 [info] Converting Potential winter distribution of roe deer - Provincial Wildlife Plan 2010 to parquet (link https://siatservices.provincia.tn.it/idt/vector/p_TN_65fd7b3f-970d-4c11-9691-8963331ed264.zip)
> 2024-07-29 11:08:12,482 [info] Converting Real d

> 2024-07-29 11:16:51,374 [info] Run execution finished: {'status': 'completed', 'name': 'convert-list-convert-list'}
